<a href="https://colab.research.google.com/github/reason84/colab-Esmf-Fold/blob/main/Esmf_Fold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title 🔧 安装 ESMFold 必要依赖 (OpenFold & Biotite)
print("1️⃣ 正在安装最新版 ESM...")
!pip install "git+https://github.com/facebookresearch/esm.git" > /dev/null 2>&1

print("2️⃣ 正在安装 OpenFold (这步可能需要几分钟，请耐心等待)...")
# 安装 OpenFold 的特定兼容版本
!pip install "git+https://github.com/aqlaboratory/openfold.git@4b41059694619831a7db195b7e0988fc4ff3a307" > /dev/null 2>&1

print("3️⃣ 正在安装 Biotite 和其他工具...")
!pip install biotite dm-tree > /dev/null 2>&1

print("✅ 所有依赖安装完成！现在可以运行预测脚本了。")

1️⃣ 正在安装最新版 ESM...
2️⃣ 正在安装 OpenFold (这步可能需要几分钟，请耐心等待)...
3️⃣ 正在安装 Biotite 和其他工具...
✅ 所有依赖安装完成！现在可以运行预测脚本了。


In [3]:
# @title 🚀 启动 ESMFold 批量预测 (CPU版)
# @markdown 点击左边的播放键，去喝杯茶，回来收菜。

import os
import time
import zipfile

print("1️⃣ 正在安装 Meta ESMFold 官方库 (fair-esm)...")
!pip install fair-esm > /dev/null 2>&1
import torch
import esm

# ================= 配置突变体列表 (R346/F359/F441 基准) =================
# 这是您最终确认的 User_WT 序列
USER_WT_SEQ = "MYSFPNSFRFGWSQAGFQSEMGTPGSEDPNTDWYKWVHDPENMAAGLVSGDLPENGPGYWGNYKTFHDNAQKMGLKIARLNSEWSRQFPNPLPRPQNFDESKQDVTEVEINENELKRLDEYANKDALNHYREIFKDLKSRGLYFIQNMYHWPLPLWLHDPIRVRRGDFTGPSGWLSTRTVYEFARFSAYTAWKFDDLVDEYSTMNEPNVVGGLGYVGVKSGFPPGYLSFELSRRAMYNIIQAHARAYDGIKSVSKKPVGIIYANSSFQPLTDKDMEAVEMAENDNRWWFFDAIIRGEITRGNEKIVRDDLKGRLDWIGVNYYTRTVVKRTEKGYVSLGGYGHGCERNSVSLAGLPTSDFGWEFFPEGLYDVLTKYWNRYHLYMYVTENGIADDADYQRPYYLVSHVYQVHRAINSGADVRGYLHWSLADNYEWASGFSMRFGLLKVDYNTKRLYWRPSALVYREIATNGAITDEIEHLNSVPPVKPLRH"

mutants = {
    "User_WT": USER_WT_SEQ,
    # === 359位点 ===
    "Mutant_F359W": USER_WT_SEQ.replace("TSDFGW", "TSDWGW"),
    "Mutant_F359H": USER_WT_SEQ.replace("TSDFGW", "TSDHGW"),
    "Mutant_F359Y": USER_WT_SEQ.replace("TSDFGW", "TSDYGW"),
    "Mutant_F359M": USER_WT_SEQ.replace("TSDFGW", "TSDMGW"),
    "Mutant_F359Q": USER_WT_SEQ.replace("TSDFGW", "TSDQGW"),
    # === 441位点 ===
    "Mutant_F441W": USER_WT_SEQ.replace("SMRFGL", "SMRWGL"),
    "Mutant_F441H": USER_WT_SEQ.replace("SMRFGL", "SMRHGL"),
    "Mutant_F441Y": USER_WT_SEQ.replace("SMRFGL", "SMRYGL"),
    "Mutant_F441L": USER_WT_SEQ.replace("SMRFGL", "SMRLGL"),
    # === 346位点 ===
    "Mutant_R346W": USER_WT_SEQ.replace("GCERNS", "GCEWNS"),
    "Mutant_R346F": USER_WT_SEQ.replace("GCERNS", "GCEFNS"),
    "Mutant_R346E": USER_WT_SEQ.replace("GCERNS", "GCEENS"),
    # === 组合突变 ===
    "Mutant_F359W_F441W": USER_WT_SEQ.replace("TSDFGW", "TSDWGW").replace("SMRFGL", "SMRWGL"),
    "Mutant_F359H_F441W": USER_WT_SEQ.replace("TSDFGW", "TSDHGW").replace("SMRFGL", "SMRWGL"),
    "Mutant_F359W_F441H": USER_WT_SEQ.replace("TSDFGW", "TSDWGW").replace("SMRFGL", "SMRHGL"),
}
# ====================================================================

OUTPUT_DIR = "mutants_pdb"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("2️⃣ 加载模型 (首次运行需下载权重，约2GB)...")
model = esm.pretrained.esmfold_v1()
model = model.eval().cpu() # 强制 CPU 模式

print(f"3️⃣ 开始批量预测 {len(mutants)} 个结构...")
print("⚠️ 提示：CPU模式下，每个蛋白约需 5-10 分钟。请保持页面开启。")

total = len(mutants)
for i, (name, seq) in enumerate(mutants.items(), 1):
    output_path = os.path.join(OUTPUT_DIR, f"{name}.pdb")
    if os.path.exists(output_path):
        print(f"[{i}/{total}] {name} 已存在，跳过。")
        continue

    print(f"[{i}/{total}] 正在折叠: {name} ...")
    try:
        with torch.no_grad():
            output = model.infer_pdb(seq)
        with open(output_path, "w") as f:
            f.write(output)
        print(f"   ✅ 完成")
    except Exception as e:
        print(f"   ❌ 失败: {e}")

# 打包下载
print("4️⃣ 正在打包...")
zip_filename = "Ginseng_Mutants_PDB.zip"
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for root, dirs, files in os.walk(OUTPUT_DIR):
        for file in files:
            zipf.write(os.path.join(root, file), file)

from google.colab import files
print(f"🎉 全部完成！正在自动下载 {zip_filename} ...")
try:
    files.download(zip_filename)
except:
    print(f"自动下载失败，请在左侧文件栏手动下载 {zip_filename}")

1️⃣ 正在安装 Meta ESMFold 官方库 (fair-esm)...
2️⃣ 加载模型 (首次运行需下载权重，约2GB)...


ModuleNotFoundError: No module named 'openfold'